In [1]:
import json
import os
import pandas as pd
import numpy as np

In [7]:
dpath = "..\\.\\data\\redial_dataset"

with open(os.path.join(dpath, "train_data.jsonl"), "r") as f:
    train_data = [json.loads(line) for line in f]
with open(os.path.join(dpath, "test_data.jsonl"), "r") as f:
    test_data = [json.loads(line) for line in f]
with open(os.path.join(dpath, "movies_with_mentions.csv"), "r") as f:
    movies = pd.read_csv(f)
movie2id = {row["movieName"]: str(row["movieId"]) for _, row in movies.iterrows()}
id2movie = {str(row["movieId"]): row["movieName"] for _, row in movies.iterrows()}

print(train_data[0].keys())
print(json.dumps(train_data[0], indent=2))

dict_keys(['movieMentions', 'respondentQuestions', 'messages', 'conversationId', 'respondentWorkerId', 'initiatorWorkerId', 'initiatorQuestions'])
{
  "movieMentions": {
    "203371": "Final Fantasy: The Spirits Within (2001)",
    "84779": "The Triplets of Belleville (2003)",
    "122159": "Mary and Max (2009)",
    "151313": "A Scanner Darkly  (2006)",
    "191602": "Waking Life (2001)",
    "165710": "The Boss Baby (2017)"
  },
  "respondentQuestions": {
    "203371": {
      "suggested": 1,
      "seen": 0,
      "liked": 1
    },
    "84779": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "122159": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "151313": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "191602": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "165710": {
      "suggested": 1,
      "seen": 0,
      "liked": 1
    }
  },
  "messages": [
    {
      "timeOffset": 0,
      "text": "H

In [28]:
conv_movies = []
for conv in train_data + test_data:
    if isinstance(conv["movieMentions"], dict):
        conv_movies.append(set(conv["movieMentions"].keys()))
    elif isinstance(
        conv["movieMentions"], list
    ):  # moviementions中若对话未提及电影，则为空列表，而不是空字典
        if len(conv["movieMentions"]) != 0:
            print(f"No empty List detected {conv}")
            raise ValueError("movieMentions is a list")
conv_movies = set().union(*conv_movies)
len_conv_movies = len(conv_movies)  # 6638
print(f"Number of unique movies mentioned in conversations: {len_conv_movies}")

len_movie_map = len(movie2id)  # 6917
print(f"Number of unique movies in movie map: {len_movie_map}")
# 提及电影多于对话中出现的电影

Number of unique movies mentioned in conversations: 6638
Number of unique movies in movie map: 6917


In [31]:
# 检查是否有对话中提及的电影不在movie map中
missing_movies = len_conv_movies
for m in conv_movies:
    if m not in movie2id:
        # print(f"Missing movie in movie map: {m}")
        missing_movies -= 1
print(f"Number of movies mentioned in conversations and present in movie map: {missing_movies}")
# 无缺失电影

Number of movies mentioned in conversations and present in movie map: 0


In [33]:
# 模态数据检查
audio_emb_files = os.listdir(os.path.join(dpath, "audio_emb"))
img_emb_files = os.listdir(os.path.join(dpath, "img_emb"))
txt_emb_files = os.listdir(os.path.join(dpath, "txt_emb"))
vdo_emb_files = os.listdir(os.path.join(dpath, "vdo_emb"))
missing_emb = []
for m in id2movie:
    if (
        f"{m}.pt" not in audio_emb_files
        or f"{m}.pt" not in img_emb_files
        or f"{m}.pt" not in txt_emb_files
        or f"{m}.pt" not in vdo_emb_files
    ):
        missing_emb.append(m)
print(f"Number of movies missing any modality embedding: {len(missing_emb)}")

Number of movies missing any modality embedding: 58
